# Modelovanie sekvenčných dát pomocou rekurentných neurónových sietí

Doterajšie neurónové siete, ktoré sme trénovali, boli všetky založené na viacvrstvovom perceptróne, ktorý sa dokáže učiť iba v prípade, ak trénovacie dáta sú nezávislé a pochádzajú z rovnakej distribúcie. To znamená, že pri trénovaní nezáleží na poradí ich použitia. Takýto prístup sme vedeli rozšíriť aj na obrazové dáta, kde sme použili konvolúciu pre extrahovanie príznakov. Čo by sa ale stalo, keby sme chceli spracovať video pomocou neurónovej siete?

Video sa síce skladá zo snímok, tie však nie sú nezávislé a na ich poradí celkom jednoznačne záleží. Na dnešnom cvičení sa pozrieme na ďalší typ neurónových sietí - rekurentných NN - ktoré dokážu pracovať s takýmito **sekvenčnými dátami**. Použijeme pritom ako vstup text, a ukážeme si dva základné prípady použitia.

## Sekvenčné dáta

Za sekvenčné dáta považujeme ľubovoľnú sadu dát, pri ktorej záleží na poradí prvkov, teda dataset vieme popísať iba v kontexte nejakej postupnosti, všeobecne: $(x^{(1)}, x^{(2)}, ..., x^{(T)})$. Táto postupnosť môže byť časová (časové rady), alebo sa môže jednať o bežnú sekvenciu údajov, ako napríklad pri prirodzenom jazyku, alebo pri sekvenovaní genómu, prípadne DNA. Typickým príkladom časových radov sú hodnoty akcií na burze.

Pri sekvenčných dátach výstup je ovplyvňovaný nielen posledným údajom, ale aj postupnosťou, ktorá predchádzala poslednému vstupu. Predstavte si pohyb guličky na obraze; na predikciu ďalšej pozície vám vo väčšine prípadov nestačí poznať jej poslednú polohu, ale potrebujete vedieť aj predošlé polohy, aby ste vedeli urobiť presnú predikciu jej pohybu.

Na druhej strane, nie v každom prípade existuje jedinečný výstup pre každý vstupný údaj, vo všeobecnosti poznáme hneď niekoľko typov úloh pri práci so sekvenčnými údajmi:

* **many-to-one** - vstupom je sekvencia, ale výstupom je jediný vektor s istou dĺžkou, prípadne iba jedno číslo. Príkladom je analýza sentimentu v texte.
* **one-to-many** - vstupom je jediná hodnota, a my chceme z nej vygenerovať postupnosť údajov. Príkladom je generovanie popisky k obrazu.
* **many-to-many** - vstup aj výstup je sekvencia. Túto kategóriu vieme ďalej deliť podľa toho, či sa výstup generuje so skĺzom alebo okamžite. Synchronizovaná predikcia je napríklad klasifikácia videa, kde olabelujeme každú snímku. Predikcia so skĺzom môže zahŕňať strojový preklad, kde preklad textu nemôžeme začať riešiť okamžite.

## Spracovanie prirodzeného jazyka

Veľké množstvo textových dát môže slúžiť ako zdroj cenných informácií a práve preto sa v posledných rokoch **spracovanie prirodzeného jazyka** (*natural language processing* - NLP) stalo dôležitou podoblasťou strojového učenia. Na nasledujúcich cvičeniach sa pozrieme na základné princípy NLP a úlohy špecifické pre texty. Na dnešnom cvičení si ukážeme príklad takzvanej analýzy sentimentu, teda kategorizácie textu podľa prístupu a názoru autora. Budeme pracovať s datasetom IMDb, ktorý obsahuje 50 000 recenzií filmov, ktoré sú zaradené do dvoch kategórií: pozitívnych (viac ako 6 hviezdičiek) a negatívnych (menej ako 5 hviezdičiek). Po stiahnutí datasetu nás čakajú úlohy ako:

1. predspracovanie dát;
2. vektorizácia textových údajov;
3. trénovanie modelu strojového učenia pre klasifikáciu;
4. práca s veľkými textovými dátami;
5. estimácia obsahu textu.

### 1. Predspracovanie dát

[Dataset si môžete stiahnuť z tohto odkazu](http://ai.stanford.edu/~amaas/data/sentiment/), následne si rozbaľte súbor (cca 80 MB).

Môžete si všimnúť, že dáta sú rozdelené do dvoch adresárov na trénovanie a testovanie, a v rámci týchto priečinkov nájdeme veľa súborov. Pre pohodlnejšiu prácu si tieto dáta nakopírujeme do jedného CSV súboru (proces môže trvať niekoľko minút). Ak sa vám nechce čakať na výsledky, [môžete si stiahnuť hotový CSV súbor](lab07/movie_data.csv) (cca 64 MB).

Rozbalenie môžete urobiť priamo v Pythone, čo by mohlo byť rýchlejšie:

In [ ]:
import tarfile
with tarfile.open("lab07/aclImdb_v1.tar.gz", 'r:gz') as tar:
    tar.extractall()

In [ ]:
import os
import sys

import pandas as pd
# pip install pyprind
import pyprind

BASEPATH = "lab07/aclImdb"

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
for subdir in ('test', 'train'):
    for cat in ('pos', 'neg'):
        path = os.path.join(BASEPATH, subdir, cat)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[cat]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

Pred ukladaním načítaných dát si ich pomiešame v náhodnom poradí, a následne ich uložíme do CSV súboru (ak ste si stiahli hotový súbor, tieto kroky môžete vynechať).

In [ ]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv("movie_data.csv", index=False, encoding='utf-8')

Následne si dataset znova načítame (tento krok už potrebujete urobiť), trošku upravíme, a skontrolujeme množstvo a obsah dát:

In [ ]:
df = pd.read_csv("movie_data.csv", encoding='utf-8')
# krok potrebny na niektorych pocitacoch
# df = df.rename(columns={"0": "review", "1": "sentiment"})
print(df.shape)
print(df.head(3))

### 2. Vektorizácia textových údajov

Neurónové siete, ale aj ďalšie algoritmy strojového učenia boli navrhnuté tak, aby dokázali pracovať s číselnými dátami, čo sa nedá povedať o texte. Práve preto v prípade textu a iných kategorických údajov je potrebné tieto údaje pretransformovať do číselnej reprezentácie. V tomto kroku použijeme prístup **bag-of-words**, ktorý každému slovu pridelí jedinečný príznakový vektor. Tento proces sa uskutoční v dvoch krokoch:

1. vytvoríme si zásobu jedinečných tokenov - napríklad slov - zo všetkých dokumentov.
2. zostrojíme príznakový vektor z každého dokumentu, kde vektor obsahuje informáciu o tom, koľkokrát sa dané slovo vyskytuje v danom dokumente.

Je jasné, že väčšina hodnôt vo vektoroch bude 0, t.j. vektory budú **sparse**, čo je presne to, čo potrebujeme.

#### 2.1. Generovanie príznakových vektorov

Pre generovanie vektorov použijeme knižnicu `scikit-learn`, ktorá je súčasťou inštalácie Anaconda. Proces si ukážeme na jednoduchých dátach, a neskôr ho aplikujeme na náš dataset.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['Roses are red',
                 'Violets are blue',
                 'Roses are read, violets are blue, wine costs less than dinner for two'])
bag = count.fit_transform(docs)

Následne si môžeme skontrolovať a analyzovať obsah vygenerovaných vektorov:

In [ ]:
print(count.vocabulary_)

In [ ]:
print(bag.toarray())

Vygenerovaná zásoba slov reprezentuje index daného slova vo vektorovej reprezentácii a vektory obsahujú počet výskytov daného slova vo vete. Tento počet nazývame aj ako **raw term frequencies**: *tf(t, d)* - počet výskytov výrazu *t* v dokumente *d*. Na poradí týchto výrazov nám nezáleží, ich poradie je odvodené z indexov zásoby slov (zvyčajne podľa abecedy).

**Poznámka**: V našom bag-of-words modeli sme použili 1-gram (unigram) model, ale existujú aj iné reprezentácie, kde sa jeden výraz skladá z viacerých tokenov, napríklad bigram: *roses are*, *are red*. Rôzne úlohy vyžadujú rôznu aritu reprezentácie.

Nevýhoda TF reprezentácie je, že sa niektoré slová často vyskytujú v príkladoch oboch typov (pozitívne a negatívne), a práve preto zvyčajne nemajú veľkú výpovednú hodnotu pre klasifikáciu. Namiesto toho teda, aby sme brali do úvahy ich surovú početnosť v dátach, môžeme použiť techniku **term frequency-inverse document frequency**: $$tf{\text -}idf(t, d) = tf(t, d) \times idf(t, d),$$

kde

$$
idf(t,d) = log\frac{n_{d}}{1 + df(d, t)}.
$$

$n_{d}$ je celkový počet dokumentov, a $df(d, t)$ je počet dokumentov *d*, ktoré obsahujú výraz *t*. Konkrétne implementácie v knižnici `scikit-learn` fungujú s menšími zmenami, ale to nás nemusí zaujímať.

Našu reprezentáciu vieme prekonvertovať do TF-IDF formy pomocou kódu:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

#### 2.2 Čistenie dát

Skutočné textové dáta často obsahujú špeciálne znaky, ktoré nemajú žiadnu výpovednú hodnotu, a práve preto by bolo vhodné ich vymazať. Zoberte príklad z jednej recenzie:

In [ ]:
print(df.loc[0, 'review'][-50:])

Text obsahuje HTML markupy, ako aj interpunkčné znamienka. V niektorých prípadoch síce interpunkcia zohráva veľkú rolu pri vyhodnocovaní textu, v našom prípade ju však nepotrebujeme, takže odstránime ju spolu s HTML tagmi. K tomu ešte spracujeme aj smajlíky ako bonus.

In [ ]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [ ]:
print(preprocessor(df.loc[0, 'review'][-50:]))
print(preprocessor("</a>This :) is :( a test :-)!"))

Predspracovanie teraz už vieme aplikovať na našich skutočných dátach:

In [ ]:
df['review'] = df['review'].apply(preprocessor)

#### 2.3. Tokenizácia dokumentov

Prvý krok pri spracovaní textu je jeho rozdelenie na menšie bloky, tzv. tokeny, ktoré väčšinou sú slová. Základným spôsobom rozdelenia do slov je rozdelenie viet podľa medzier:

In [ ]:
def tokenizer(text):
    return text.split()

print(tokenizer("runners like running and thus they run"))

#### 2.4. Stemming (a lematizácia)

Ako môžete vidieť na príklade vyššie, niektoré slová (ako *running* a *run*, sčasti aj *runners*) reprezentujú podobný koncept, práve preto je zbytočné ich mať viackrát v zásobe slov. Takisto by sme potrebovali spracovať množné čísla a previesť slová do singulárneho tvaru. Tento proces sa nazýva **stemming** a existuje niekoľko algoritmov na jeho realizáciu, jeden zo základných je **Porter stemming**, ktorý je dostupný v knižnici Natural Language Toolkit (súčasťou Anacondy):

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer_porter("runners like running and thus they run"))

Na vyššom príklade vidíte efekt stemmingu, ktorý ale nie je dokonalý proces (napríklad *thus* nesprávne zjednodušil na tvar *thu*). Druhý podobný proces - lematizácia - vám vždy vráti základný slovníkový tvar slova, je však výpočtovo náročnejší aj keď zvyčajne dáva lepšie výsledky.

#### 2.5. Odstránenie stop slov

Stop slová sú slová, ktoré sa často vyskytujú vo všetkých textoch a práve preto ich nemá veľký zmysel spracovať, keďže nemajú veľkú informačnú hodnotu pre klasifikáciu alebo inú úlohu strojového učenia. Síce *tf-idf* čiastočne eliminuje potrebu odstránenia takýchto slov, keďže zmenší dôležitosť často sa opakujúcich slov, v niektorých prípadoch je dôležité tieto slová odstrániť, na čo existujú hotové množiny pre rôzne jazyky.

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
print([w for w in tokenizer_porter("a runner likes running and runs a lot") if w not in stop])

#### 2.6. Príprava na trénovanie

Pred tým, než použijeme neurónové siete, proces trénovania klasifikačného modelu si ukážeme pomocou logistickej regresie. Pri trénovaní použijeme aj optimalizáciu hyperparametrov pomocou `sklearn`. Najprv si ale pripravíme trénovacie a testovacie dáta:

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

Následne si natrénujeme model:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    }
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy', cv=5,
                           verbose=2, n_jobs=1)  # n_jobs=-1 for parallel processing
gs_lr_tfidf.fit(X_train, y_train)

Najúspešnejšie nastavenie parametrov a príslušné výsledky nájdeme pomocou kódu:

In [ ]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

Môžeme to overiť aj na testovacích dátach:

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

## Rekurentné neurónové siete

V doterajších dopredných štruktúrach sme mali jasnú postupnosť krokov, kde na základe vstupu $x$ sa vygenerovala najprv aktivácia skrytej vrstvy $h$, a následne výstup siete $o$. Rekurentné siete zakomponujú aj časový údaj, a vstup do skrytého bloku je rozšírený o predošlý stav siete, ako je to znázornené na obrázku nižšie.

![](lab07/rnn-scheme.jpg)

Spôsob výpočtu výstupu je úplne rovnaký ako v predošlých prípadoch, až na to, že máme tu časom rozšírenú štruktúru:

![](lab07/rnn-activation.jpg)

Pracujeme teda s tromi maticami váh: $w_{xh}$ - váhy smerujúce od vstupu do skrytej vrstvy; $w_{hh}$ - váhy rekurentného bloku; $w_{ho}$ - váhy smerujúce od rekurentného bloku do výstupu. Výpočty potom prebiehajú nasledovne (všetky hodnoty sú reálne matice alebo vektory):

1. najprv sa vypočíta predaktivácia rekurentného bloku
    $$sum_{h}^{(t)} = w_{xh} x^{(t)} + w_{hh} h^{(t-1)} + b_h$$
2. v ďalšom kroku vypočítame aktiváciu rekurentného bloku
    $$act_{h}^{(t)} = act( sum_{h}^{(t)} ) = act( w_{xh} x^{(t)} + w_{hh} h^{(t-1)} + b_h )$$
3. výstup siete sa vypočíta podľa štandardných procesov
    $$act_{o}^{(t)} = act( w_{ho} h^{(t)} + b_o )$$

In [ ]:
import torch
import torch.nn as nn

torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True) 

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

In [ ]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

## output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print('   Input           :', xt.numpy())
    
    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh    
    print('   Hidden          :', ht.detach().numpy())
    
    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.detach().numpy())
    print('   RNN output      :', output[:, t].detach().numpy())
    print()

Pri výpočte chyby musíme zohľadniť časovú závislosť:

$$L=\sum_{t=1}^{T} L^{(t)}$$

Pri učení pomocou backpropagation potom:

$$\frac{\delta L^{(t)}}{\delta \textbf{W}_{hh}} = \frac{\delta L^{(t)}}{\delta \textbf{o}^{(t)}} \times \frac{\delta \textbf{o}^{(t)}}{\delta \textbf{h}^{(t)}} \times \left ( \sum_{k=1}^{t} \frac{\delta \textbf{h}^{(t)}}{\delta \textbf{h}^{(k)}} \times \frac{\delta \textbf{h}^{(k)}}{\delta \textbf{W}_{(hh)}} \right )$$

pričom

$$\frac{\delta \textbf{h}^{(t)}}{\delta \textbf{h}^{(k)}} = \prod_{i=k+1}^{t} \frac{\delta \textbf{h}^{(i)}}{\delta \textbf{h}^{(i-1)}}$$

Ďalšie variácie rekurentných blokov sú output-to-hidden, resp. output-to-output.

Keďže pri backprope sa použije súčin jednotlivých komponentov, môžu nastať tri situácie:

1. váhy v rekurentnom bloku sú menšie ako jeden - vanishing gradient;
2. váhy v rekurentnom bloku sú vyššie ako jeden - exploding gradient;
3. váhy v rekurentnom bloku sú jeden - ideálny prípad.

Okrem udržiavania váh v okolí 1, ďalšie možnosti riešenia gradientového problému sú:

* gradient clipping - nastavíme minimálnu resp. maximálnu hodnotu gradientu, mimo ktorého intervalu gradient vynulujeme;
* truncated backpropagation through time - nastavíme maximálny počet časových cyklov, ktoré ovplyvňujú gradient;
* LSTM siete.

## Long Short-Term Memory siete

Základom LSTM sietí je **memory cell**, ktorá nahrádza skrytú vrstvu štandardnej RNN. Každá bunka obsahuje rekurentnú hranu s váhou 1, hodnoty spojené s touto hranou reprezentujú stav bunky. Celková štruktúra bunky potom vyzerá nasledovne:

![](lab07/lstm.jpg)

Ako aj blokový model naznačuje, stav bunky ($c^{(t)}$) je upravovaný zložitejším procesom, a skladá sa z niekoľkých krokov (niekedy nazývané *gate*-y). Tieto gate-y určujú, kedy ktorá informácia má byť zahodená. Bunka okrem toho naďalej má aj skrytý stav ($h^{(t)}$). Jednotlivé gate-y potom fungujú nasledovne:

* **forget gate** - zabezpečuje, aby si bunka mohla reinicializovať svoju pamäť, tým pádom hodnota nenarastá nekonečne (určuje, ktorá hodnota sa zapamätá).
$$f_{t} = \sigma \left ( \textbf{W}_{xf} \textbf{x}^{(t)} + \textbf{W}_{hf} \textbf{h}^{(t-1)} + \textbf{b}_{f} \right )$$
* **input gate** - aktualizuje stav bunky spolu s kandidátnou hodnotou:
$$i_{t} = \sigma \left ( \textbf{W}_{xi} \textbf{x}^{(t)} + \textbf{W}_{hi} \textbf{h}^{(t-1)} + \textbf{b}_{i} \right )$$
$$\tilde{C}_{t} = \tanh \left ( \textbf{W}_{xc} \textbf{x}^{(t)} + \textbf{W}_{hc} \textbf{h}^{(t-1)} + \textbf{b}_{c} \right )$$

z toho

$$C^{(t)} = (C^{(t-1)} \odot f_{t}) \oplus (i_t \odot \tilde{C}_{t})$$

* **output gate** - rieši aktualizáciu hodnôt skrytých neurónov:
$$o_{t} = \sigma \left ( \textbf{W}_{xo} \textbf{x}^{(t)} + \textbf{W}_{ho} \textbf{h}^{(t-1)} + \textbf{b}_{o} \right )$$

z toho

$$h^{(t)} = o_t \odot \tanh (C^{(t)})$$

## Príklady

Príklady na využitie rekurentných sietí nájdete [tu pre analýzu sentimentu](https://github.com/rasbt/machine-learning-book/blob/main/ch15/ch15_part2.ipynb) (rovnaká úloha ako na cvičení) a [tu pre modelovanie jazyka a generovanie textu](https://github.com/rasbt/machine-learning-book/blob/main/ch15/ch15_part3.ipynb).

## Použité zdroje

* Blog, Andrej Karpathy. "The Unreasonable Effectiveness of Recurrent Neural Networks." URL: http://karpathy.github.io/2015/05/21/rnn-effectiveness/ dated May 21 (2015): 31.
* Werbos, Paul J. "Backpropagation through time: what it does and how to do it." Proceedings of the IEEE 78, no. 10 (1990): 1550-1560.
* Pascanu, Razvan, Tomas Mikolov, and Yoshua Bengio. "On the difficulty of training recurrent neural networks." In International conference on machine learning, pp. 1310-1318. Pmlr, 2013.
* Memory, Long Short-Term. "Long short-term memory." Neural computation 9, no. 8 (2010): 1735-1780.
* Gers, Felix A., Jürgen Schmidhuber, and Fred Cummins. "Learning to forget: Continual prediction with LSTM." Neural computation 12, no. 10 (2000): 2451-2471.
* Chung, Junyoung, Caglar Gulcehre, KyungHyun Cho, and Yoshua Bengio. "Empirical evaluation of gated recurrent neural networks on sequence modeling." arXiv preprint arXiv:1412.3555 (2014).

* Raschka, Sebastian, Yuxi Hayden Liu, Vahid Mirjalili, and Dmytro Dzhulgakov. Machine Learning with PyTorch and Scikit-Learn: Develop machine learning and deep learning models with Python. Packt Publishing Ltd, 2022. Kapitola 15